In [1]:
#I'll point to a file
#I'll parse out the file name and get the start and end date

#I'll parse out the int16 percision

#the output will be a dataframe with the timestamp for each sample and the float

#check that the last timestamp is the same as the end date

#graph the data

#install the wvunpack tool
#sudo apt-get install wvunpack

In [2]:
from datetime import datetime, timezone

descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_acceleration_mDsE2_int16-f6_1_32hz"

interval_s = 1/float(descriptor.split("_")[-1][:-2])
data_type_descriptor = descriptor.split("_")[-3]
num_channels = 3
endianness = "le"


float_scale = 1
if "-" in data_type_descriptor:
    np_type = data_type_descriptor.split("-")[0]
    float_scale = 2 ** int(data_type_descriptor.split("-")[1][1:])
else:
    np_type = data_type_descriptor


file_base_path = "/home/chowder/Downloads/"
file_name = "20251218T170831p000Z_20251218T171314p343Z.wv"
full_file_path = file_base_path + file_name


start_dt = datetime.strptime(file_name.split("_")[0], "%Y%m%dT%H%M%Sp%fZ").astimezone(timezone.utc)
end_dt = datetime.strptime(file_name.split("_")[1], "%Y%m%dT%H%M%Sp%fZ.wv").astimezone(timezone.utc)

In [6]:
import subprocess
import numpy as np
import pandas as pd
import os

#check that the file exists
if not os.path.exists(full_file_path):
    raise FileNotFoundError(f"File not found: {full_file_path}")
else:
    print(f"File found: {full_file_path}")

# Decompress the wavpak file to raw PCM data
# Use --raw to get raw PCM output, and -o - to output to stdout
wvunpack_cmd = ["wvunpack", "--raw", full_file_path, "-o", "-"]
result = subprocess.run(wvunpack_cmd, capture_output=True, check=True)
raw_data = result.stdout

# Convert binary data to numpy array
# int16 little-endian: '<i2'
dtype = '<i2'  # int16 little-endian
data_array = np.frombuffer(raw_data, dtype=dtype)

# Reshape to (n_samples, num_channels) for X, Y, Z
n_samples = len(data_array) // num_channels
data_reshaped = data_array[:n_samples * num_channels].reshape(n_samples, num_channels)

# Scale the data using float_scale
data_scaled = data_reshaped.astype(float) / float_scale

# Create timezone-aware datetime index
sampleDT = pd.date_range(start=start_dt, periods=n_samples, freq=pd.Timedelta(seconds=interval_s), tz='UTC')

# Create DataFrame with sampleDT as index and X, Y, Z as columns
df = pd.DataFrame(data_scaled, columns=['X', 'Y', 'Z'], index=sampleDT)
df.index.name = 'sampleDT'

print(f"DataFrame shape: {df.shape}")
print(f"Time range: {df.index[0]} to {df.index[-1]}")
print(f"\nFirst few rows:")
print(df.head())


File found: /home/chowder/Downloads/20251218T170831p000Z_20251218T171314p343Z.wv
DataFrame shape: (9068, 3)
Time range: 2025-12-19 00:08:31+00:00 to 2025-12-19 00:13:14.343750+00:00

First few rows:
                                    X    Y         Z
sampleDT                                            
2025-12-19 00:08:31+00:00         0.0  0.0  0.140625
2025-12-19 00:08:31.031250+00:00  0.0  0.0  0.140625
2025-12-19 00:08:31.062500+00:00  0.0  0.0  0.140625
2025-12-19 00:08:31.093750+00:00  0.0  0.0  0.140625
2025-12-19 00:08:31.125000+00:00  0.0  0.0  0.140625


In [7]:

df


,X,Y,Z
sampleDT,,,
2025-12-19 00:08:31+00:00,0.0,0.0,0.140625
2025-12-19 00:08:31.031250+00:00,0.0,0.0,0.140625
2025-12-19 00:08:31.062500+00:00,0.0,0.0,0.140625
2025-12-19 00:08:31.093750+00:00,0.0,0.0,0.140625
2025-12-19 00:08:31.125000+00:00,0.0,0.0,0.140625
...,...,...,...
2025-12-19 00:13:14.218750+00:00,0.0,0.0,0.140625
2025-12-19 00:13:14.250000+00:00,0.0,0.0,0.140625
2025-12-19 00:13:14.281250+00:00,0.0,0.0,0.140625
